# Collecting description of offers

In [ ]:
df=pd.read_parquet('df.gzip')

In [ ]:
df.index

In [ ]:
df_desc_read.index

In [ ]:
desc_indices=df.index[~df.index.isin(df_desc_read.index)]
desc_indices

In [ ]:
desc_indices.drop_duplicates()
desc_indices.shape

In [ ]:
collector_description={}

for i in range(len(desc_indices)):
    base_url = 'https://www.sreality.cz/api/cs/v2/estates/{}'.format(desc_indices[i])

    r = requests.get(base_url)

    #print("starting sleep")
    #sleep(randint(0,2))

    if r.status_code==404:
        break
    
    elif r.status_code==200:
        r_dict=r.json()
        r_dict["hash_id"]=desc_indices[i]

        collector_description[i]=r_dict
        
        print(f"Description {i} was scraped.")

In [ ]:
description_individual = {}

for page, r_dict in collector_description.items():
        
        info_relevant = pd.Series(dtype="object")
        info_relevant["description"]=r_dict["text"]["value"]

        r_dict_values=pd.DataFrame(r_dict["items"], columns =['type', 'name', 'value'])

        info_relevant["price"]=r_dict_values["value"][r_dict_values["name"]=="Cena"]
        info_relevant["price_note"]=r_dict_values["value"][r_dict_values["name"]=="Poznámka k ceně"]
        info_relevant["price_per_sqm"]=r_dict_values["value"][r_dict_values["name"]=="Cena za m²"]
        info_relevant["id_order"]=r_dict_values["value"][r_dict_values["name"]=="ID zakázky"]
        info_relevant["date_update"]=r_dict_values["value"][r_dict_values["name"]=="Aktualizace"]
        info_relevant["building_type"]=r_dict_values["value"][r_dict_values["name"]=="Stavba"]
        info_relevant["availability"]=r_dict_values["value"][r_dict_values["name"]=="Stav"]
        info_relevant["building_condition"]=r_dict_values["value"][r_dict_values["name"]=="Stav objektu"]
        info_relevant["building_location"]=r_dict_values["value"][r_dict_values["name"]=="Umístění objektu"]
        info_relevant["building_floor_type"]=r_dict_values["value"][r_dict_values["name"]=="Typ domu"]
        info_relevant["building_floor"]=r_dict_values["value"][r_dict_values["name"]=="Podlaží"]
        info_relevant["date_moving_in"]=r_dict_values["value"][r_dict_values["name"]=="Datum nastěhování"]
        info_relevant["area_build_up"]=r_dict_values["value"][r_dict_values["name"]=="Plocha zastavěná"]
        info_relevant["garage"]=r_dict_values["value"][r_dict_values["name"]=="Garáž"]
        info_relevant["parking"]=r_dict_values["value"][r_dict_values["name"]=="Parkování"]

        if sum(r_dict_values['name'] == "Voda")>0:
                index_nr=int(r_dict_values.index[r_dict_values['name'] == "Voda"].tolist()[0])
                r_dict_values["value"][r_dict_values["name"]=="Voda"][index_nr][0]["value"]
                del index_nr

        if sum(r_dict_values['name'] == "Topení")>0:
                index_nr=int(r_dict_values.index[r_dict_values['name'] == "Topení"].tolist()[0])
                r_dict_values["value"][r_dict_values["name"]=="Topení"][index_nr][0]["value"]
                del index_nr

        if sum(r_dict_values['name'] == "Doprava")>0:
                index_nr=int(r_dict_values.index[r_dict_values['name'] == "Doprava"].tolist()[0])
                r_dict_values["value"][r_dict_values["name"]=="Doprava"][index_nr][0]["value"]
                del index_nr

        if sum(r_dict_values['name'] == "Telekomunikace")>0:
                index_nr=int(r_dict_values.index[r_dict_values['name'] == "Telekomunikace"].tolist()[0])
                r_dict_values["value"][r_dict_values["name"]=="Telekomunikace"][index_nr][0]["value"]
                del index_nr

        if sum(r_dict_values['name'] == "Plyn")>0:
                index_nr=int(r_dict_values.index[r_dict_values['name'] == "Plyn"].tolist()[0])
                r_dict_values["value"][r_dict_values["name"]=="Plyn"][index_nr][0]["value"]
                del index_nr

        info_relevant["energy_efficient_rating"]=r_dict_values["value"][r_dict_values["name"]=="Energetická náročnost budovy"]
        info_relevant["lift"]=r_dict_values["value"][r_dict_values["name"]=="Výtah"]
        info_relevant["equipped"]=r_dict_values["value"][r_dict_values["name"]=="Vybavení"]

        description_individual[r_dict["hash_id"]] = info_relevant

In [ ]:
r_dict_values

In [ ]:
description_individual[r_dict["hash_id"]] = info_relevant

In [ ]:
df_desc = pd.concat(description_individual).unstack()
df_desc.head()

In [ ]:
df_desc_full = pd.concat([df_desc, df_desc_read])
df_desc_full.to_parquet('df_desc_full.gzip')  
#df_desc.to_parquet('df_desc.gzip')  

# New version of scraping - proxies and multiple requests at the same time

In [ ]:
from fp.fp import FreeProxy
import gevent
from gevent import monkey, pool
#https://stackoverflow.com/questions/15322701/gevent-pool-with-nested-web-requests
#https://pypi.org/project/free-proxy/

In [ ]:
proxy = FreeProxy(country_id=['DE', "CZ", "BR"]).get()

In [ ]:
urls=np.add("https://www.sreality.cz/api/cs/v2/estates/", desc_indices)

In [ ]:
monkey.patch_all()

jobs = []
links = []
p = pool.Pool(10)

add

urls = [
    'http://www.google.com', 
    # ... another 100 urls
]
    
def get_jsons(url):
    r = requests.get(url)
    if r.status_code == 200:
        r_dict = r.json()
        r_dict["hash_id"]=desc_indices[i]

        collector_description[i]=r_dict
        
for url in urls:
    jobs.append(p.spawn(get_jsons, url))
gevent.joinall(jobs)